# Hito 1

## 1. Preliminares

A continuación se detallan las características del problema a evaluar.

### 1.1 Problema (Análisis de sentimientos en Twitter)

El problema corresponde a uno de análisis de texto, donde queremos identificar el sentimiento asociado a un texto; el cual se refiere a un tweet.

Dado ello, es un problema de clasificación.

Objetivo: identificar si un tweet tiene polaridad positiva o negativa (sentimiento expresado).

* Variable objetivo: describe una emocion que resume el contenido de un tweet. Para conveniencia del trabajo, estas serán transformadas a emociones positivas o negativas de acuerdo a su morfología.

* Atributos: inicialmente existen textos que son tweets escritos en la plataforma de Twitter. Sin embargo, con objetivo de crear atributos para la predicción de emociones, transformaremos dichos textos en vectores de ocurrencia.

### 1.2 Modelos a implementar

Dado que el problema a estudiar es un problema de clasificación, proponemos la implementación de los siguientes modelos:

* Logistic Regression: método de clasificación que presente una fácil implementación y en general, un buen desempeño.

* KMeans: 

* Linear Discriminant Analysis: algoritmo supervisado (tenemos conocimiento previo sobre la existencia de las clases y) y generativo (buscamos aprender sobre las clases a partir de los atributos X).

* Random Forest Classifier: dado que es un buen predictor para clasificadores al promediar el desempeño de un grupo de clasificadores, e implementa un proceso de validación cruzada con un sample menor al número total de atributos disponibles; permitiendo evaluar el desempeño "out-of-bag" en cada iteración.

In [ ]:
# agregar modelos y grillas

### 1.3 Métricas y estrategia de división de muestras

In [ ]:
accuracy_score, roc_auc_score, confusion_matrix, classification_report

### 1.4 Procesamiento tentativo

## 2. Aspectos Computacionales

In [16]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.feature_extraction.text import CountVectorizer

from static import *

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 6)

## 3. Análisis Exploratorio

In [3]:
# lectura de datos
data = pd.read_csv('training_tweets.csv').drop('Unnamed: 0', axis=1)

### 3.1 limpieza de tweets

In [4]:
# aplicando normalización y lemmatizacion a los tweets
df_clean = nlp_cleaning(data)

In [5]:
df = df_clean.loc[:, ['content_clean', 'sentiment']]
df.rename(columns={'content_clean': 'content'}, inplace=True)

### 3.2 Transformando emociones

In [6]:
emotion_dict = {
    'worry': 'negativa',
    'happiness': 'positiva',
    'sadness': 'negativa',
    'love': 'positiva',
    'surprise': 'positiva',
    'fun': 'positiva',
    'relief': 'positiva',
    'hate': 'negativa',
    'empty': 'negativa',
    'enthusiasm': 'positiva',
    'boredom': 'negativa',
    'anger': 'negativa'
}

In [7]:
# reemplazando emociones
df['sentiment'] = df['sentiment'].replace(emotion_dict)

In [12]:
# reemplazando emocion neutral
binary = ['negativa', 'positiva']
df['sentiment'] = [i.replace('neutral', np.random.choice(binary, 1, p=[0.51, 0.49])[0]) for i in df['sentiment']]

### 3.3 Transformamos el conjunto de datos en una matriz que represente la ocurrencia de cada palabra

In [17]:
# Create a Vectorizer Object
vectorizer = CountVectorizer()

In [20]:
# ajustamos el modelo con los
X = vectorizer.fit_transform(df['content'])

In [28]:
df_count = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

In [65]:
# reducimos el espacio eliminando palabras insignificantes
df_count = df_count.iloc[:, 899:-5]

In [70]:
# juntamos los conjuntos de datos
df = pd.concat([df, df_count], axis=1)

In [72]:
df.head()

,content,sentiment,aaaaaaaaaamazing,aaaaaaaafternoon,aaaaaaaahhhhhhhh,aaaaaah,aaaaahhhh,aaaaall,aaaaand,aaaaaoouoouoouu,...,zstanu,zu,zuccini,zuljin,zulu,zuluxhosa,zum,zumba,zune,zwarte
0,happy mama day to all mother,positiva,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,i am lost . please help me find a good home .,negativa,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,yes yes i am networking whore to the fullest ....,positiva,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,you wish that would have been your tweet after...,positiva,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,now i am doing the microeconomics project ihat...,negativa,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
df.shape

(30000, 26833)

# 4. Feature Engineering

In [ ]:
# analisis exploratorio: palabras mas comunes y distribucion de clases en vector objetivo
# patron de datos perdidos